<div style="display: flex; justify-content: space-between; align-items: center; padding: 10 px;">
    <h3 style="margin: 0 0 20px; color: #FFFFFF; line-height: 1"> CS5002: Programming Principles and Practice</h3>
    <img src="../rsc/logo.png" alt="University Logo" style="width: 80px; height: auto;"/>
</div>
                                  <!--  -->
<div style="display: flex; justify-content: space-between; width: 100%;">
  <span style="text-align: left;">Practical P3: Data analysis and visualisation Python</span>
  <span style="text-align: right; font-size: 12px;">Student ID: 200013825</span>
</div> 


### Introduction 
This project aims to modularly read, clean and analyze real-world csv data. Below is the introduction on all the modules and major functions that worked together to complete data analysis.

`data_mining.py`: This executable python 
script automates data cleaning process, I used 
```python
invalid_mask = ~data[column].isin(valid_set)
```
to collect a list of booleans, indicating whether each csv code is admissible. 

Based on the context of this dataset, it is more efficient to neglect the type of the data when checking consistency, and then convert all data into pandas' categorical type. 
``` python
data[column] = data[column].astype(str)
```
I have converted all column in the data as string to help the determination of admissible values, and checked for missing columns or values, so then if encountered with any missing value, the row will be dropped.

To decode the csv data and to give context to the later stages of data analysis, i incorprated the data mapping at the stage of data_mining, so that the **cleaned_data.csv** contains all the code derived from json, with all the data being cleaned. I achieved this step by creating a helper function called **map_data** which takes the data to map as well as the json that was loaded in the **clean_data** function, and use **.map()** to map the data based on json if the column name match wth the json dictionary keys. Below is the logic to map data
```python
for column in data_to_map.columns:
        if column in decode_helper:
            data_to_map[column] = data_to_map[column].map(decode_helper[column])
```

`data_analysis.py`: This module contains function **analyze_data** and compute all the required analysis, then store all output from different requirements as a big dictionaries of dictionaires. This makes the overall workflow modular and clean with minimal repetition. One drawback would be the fact that all the analysis is hard-coded in, and reusability is diminished from this specific feature.

There are in total of 12 required data analysis, in the **data_analysis.py**, i have set them as requirements 1-12, and used pandas to analyze. A few analysis will be illustrated below. 

1. To get the percentage of records for any column, i used:
```python
column_counts = data['col_name'].value_counts(normalize=True) * 100
# then append the result to the big collective dictionary
results['requirement n name'] = column_counts.round(2).to_dict()
```
2. To group two columns together, i used:
```python
results['requirement n name'] = (data.groupby(['col_1', 'col_2']).size().to_dict())
```
As shown above, i collect each reqired results and have stored them in one dictionary called results. This helps with further processing and storing, as dictionary can centralize calculated results, and it can store a diverse range of data types.




### Problem Encountered

##### __1. infinite commit loop__: 
When I was trying to set up hooks for git to automatically update the gitlog everytime after commit, i accidently created an infinite loop of git commit, below was what I have included in the post-commit* file in .git/hooks:

``` python
#!/bin/bash
git log --oneline  --graph  > gitlog.txt
git commit -m 'gitlog.txt is updated with the latest commit history'
```
and this created redundant git log histroy that are shown in the gitlog.txt

I fixed it by deleting the line to commit within post-commit, after i realized that adding commit within post-commit can trigger recursive calls.

##### __2. system path append__: 
When I was trying to append certain relavant path to the sys path in order for jupyter notebook to read the file, i added the file path to sys path. After I print sys.path to check, i saw multiple paths being appended in the system paths, in order to clean this, i imported `site` which is a package that can help clear all custom paths and only keep the default path. I then used 
```python
sys.path = list(site.getsitepackages()) + sys.path [:1]
```
to implement this strategy. 
After clearing the system path, i appended the relative path using sys.path.append, and successfully imported the modules from **code** folder. As shown below.
```python
sys.path.append('./code')
```
Certain compiled python file was created to help faster import, and such files were added to .gitignore to prevent unncessary commiting and pushing because **\__pycache__\** will be created everytime the imported modules are ran.

##### __3. using venv in jupyter notebook__: 


##### __4. html header editing__: 
When i was trying to create proffesional looking header using html, it worked perfectly in vscode like so:
***
![Image](../rsc/Jupyter_heading.png)
***
But when i viewed it in jupyter notebook via jupyter, it did not look that aligned as shown below: 
***
![Image](../rsc/Jupyter_heading_notgood.png)
***





**First set up the neccessary dependencies for the project**

In [ ]:
# import pandas as pd
# import sys
# sys.path.append('./code')
# from data_analysis import analyze_csv


import sys
sys.path.append("../code")

from data_analysis import analyze_csv

results_dict = analyze_csv()



FileNotFoundError: [Errno 2] No such file or directory: './data/cleaned_data.csv'